<div class="alert alert-block alert-warning">
<b>Комментарий от ревьюера:</b> 
    
~~Мария, добрый день! Меня зовут Влада. Ниже в файле ты найдешь мои комментарии: <font color='green'>зеленый цвет — «все отлично»; </font> <font color='blue'>синий — «хорошо, но можно лучше (исправлять необязательно)»; </font> <font color='red'>красный — «нужно исправить».</font> Комментарии в самом коде я отделяю знаками «###». Пожалуйста, не удаляй мои комментарии, они мне нужны при повторной проверке.~~
    
Мария, спасибо за доработки, все в порядке!

</div>

# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

#### Задачи исследования:
1. Ознакомление с данными
 - Импортирование библиотеки
 - Предварительный просмотр данных
 - Общее получение информации для дальнейшего анализа


2. Разделить исходные данные на обучающую, валидационную и тестовую выборки.


3. Исследовать качество разных моделей, меняя гиперпараметры. Кратко написать выводы исследования.


4. Проверка качества модели на тестовой выборке.


5. Дополнительное задание: проверить модели на вменяемость.

## 1. Откройте и изучите файл

In [1]:
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
import joblib

data = pd.read_csv('/datasets/users_behavior.csv')

In [2]:
data.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [3]:
data.shape

(3214, 5)

**Вывод:** Представленные данные состоят из 5 признаков и 3214 объектов. Целевым признаком, в данном случае, является is_ultra, т.е. каким тарифом пользовался клиент в течение месяца («Ультра» — 1, «Смарт» — 0). Целевой признак является категориальным. Задача относится к классу "обучение с учителем". Далее пойдет речь о бинарной (двоичной) классификации. 

<div class="alert alert-block alert-success">
Ок, на данные посмотрели. Да, перед нами задача классификации. Целевая метрика – accuracy.
</div>

## 2. Разбить данные на выборки
1. Надо зафиксировать псевдослучайность для всех используемых в проекте алгоритмов.
2. Разбиваем представленные данные на три выборки: 60% - обучающая, 20% - валидационная, 20% - тестовая. 

In [4]:
rnd_state = 12345

In [5]:
#Бьем выборку один раз на 0.6/0.4, а потом оставшиеся 0.4 бьем по 0.5. 

data_train, data_valid = train_test_split(data, test_size=0.4, random_state=rnd_state)
data_valid, data_test = train_test_split(data_valid, test_size=0.5, random_state=rnd_state)

<div class="alert alert-block alert-danger">

~~Проверь размеры выборок, которые получились. Значения test_size следует задать другими в обоих вызовах train_test_split, чтобы пропорции получились 60%-20%-20%.~~
</div> **Исправила**

<div class="alert alert-block alert-success">
Все верно, молодец.
</div>


In [6]:
features_train = data_train.drop(['is_ultra'], axis=1) 
target_train = data_train['is_ultra']

features_valid = data_valid.drop(['is_ultra'], axis=1)
target_valid = data_valid['is_ultra']

#### Вывод

Разбили данные на выборки, получилось следующее соотношение: 60% / 20% / 20%. 
После проверки модели на валидационной выборки, проверим на тестовой.  

<div class="alert alert-block alert-success">
Хорошо, данные подготовили.
</div>

## 3. Исследовать модели

Рассмотрим три алгоритма классификации: "Дерево решений", "Модель случайного леса", "Модель Логистической регрессии". 

#### Модель "Дерево решений"

In [7]:
#Модель "Дерево решений"

for max_depth in range(1, 21, 2):
    model = DecisionTreeClassifier(random_state=rnd_state, max_depth=max_depth)
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    accuracy = accuracy_score(target_valid, predicted_valid)
    print(max_depth, accuracy)

1 0.7542768273716952
3 0.7853810264385692
5 0.7791601866251944
7 0.7822706065318819
9 0.7822706065318819
11 0.7620528771384136
13 0.7558320373250389
15 0.7465007776049767
17 0.7356143079315708
19 0.7278382581648523


**Вывод:** исходя из вылидационной выборки, мы наблюдаем, что наиболее высокую оценку правильности мы имеем при max_depth=3, accurecy = 0.7853810264385692. 

<div class="alert alert-block alert-success">
Молодец, что настраиваешь гиперпараметры, а также задаешь random_state для воспроизводимости результатов.
</div>

<div class="alert alert-block alert-info">

Лучшее значение параметра (с оптимальным качеством) лучше находить в коде, а не искать вручную, так есть шанс ошибиться.
    
Можно было дополнительно изобразить график зависимости качества от значения параметра.
</div>

#### Модель "Случайного леса"

In [8]:
#Модель "Случайного леса"
#Рассмотрим при гиперпараметре max_depth=1

for estim in range(10, 101, 10):
    model = RandomForestClassifier(n_estimators=estim, max_depth=1, random_state=rnd_state)
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    accuracy = accuracy_score(target_valid, predicted_valid)
    print(estim, accuracy)

10 0.7558320373250389
20 0.7667185069984448
30 0.7667185069984448
40 0.776049766718507
50 0.7589424572317263
60 0.7636080870917574
70 0.7636080870917574
80 0.7636080870917574
90 0.776049766718507
100 0.7744945567651633


In [9]:
#Рассмотрим при гиперпараметре max_depth=3 (как в предыдущей моделе)

for estim in range(10, 101, 10):
    model = RandomForestClassifier(n_estimators=estim, max_depth=3, random_state=rnd_state)
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    accuracy = accuracy_score(target_valid, predicted_valid)
    print(estim, accuracy)

10 0.7853810264385692
20 0.7869362363919129
30 0.7869362363919129
40 0.7869362363919129
50 0.7869362363919129
60 0.7869362363919129
70 0.7884914463452566
80 0.7884914463452566
90 0.7884914463452566
100 0.7884914463452566


In [10]:
#Рассмотрим при гиперпараметре max_depth=10
for estim in range(10, 101, 10):
    model = RandomForestClassifier(n_estimators=estim, max_depth=10, random_state=rnd_state)
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    accuracy = accuracy_score(target_valid, predicted_valid)
    print(estim, accuracy)

10 0.7916018662519441
20 0.7916018662519441
30 0.7947122861586314
40 0.7962674961119751
50 0.7931570762052877
60 0.7978227060653188
70 0.7947122861586314
80 0.7962674961119751
90 0.7947122861586314
100 0.7947122861586314


In [11]:
#Рассмотрим при гиперпараметре max_depth=None
for estim in range(10, 101, 10):
    model = RandomForestClassifier(n_estimators=estim, max_depth=None, random_state=rnd_state)
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    accuracy = accuracy_score(target_valid, predicted_valid)
    print(estim, accuracy)

10 0.7853810264385692
20 0.7869362363919129
30 0.7838258164852255
40 0.7838258164852255
50 0.7916018662519441
60 0.7853810264385692
70 0.7791601866251944
80 0.7822706065318819
90 0.7838258164852255
100 0.7853810264385692


<div class="alert alert-block alert-info">

~~Можно еще рассмотреть max_depth=None (это значение по умолчанию).~~
</div>
**Рассмотрела**

**Вывод:** Модель "Случайного леса" предсказывает тариф немного точнее, если **max_depth=10**, accuracy = **0.7978227060653188** и n_estimators=60. 

Если же мы поменяем параметр **max_depth с 10 на 3** (как в предыдущей моделе), то accuracy = **0.7884914463452566**

При **max_depth = 1**, процент предсказания наимее высокий, хотя также не сильно меньше, n_estimators=40; n_estimators=90, accuracy = **0.776049766718507.** 

При **max_depth = None**, accuracy = **0.7916018662519441**, n_estimators=50

**Остановимся на варианте: max_depth=10, accuracy = 0.7978227060653188 и n_estimators=60 с наиболее высокой долей предсказания.**

#### Модель "Логистическая регрессии"

In [12]:
#Модель "Логистическая регрессии"

def logistic_regression():
    model = LogisticRegression(solver=solver, random_state=rnd_state)
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    accuracy = accuracy_score(target_valid, predicted_valid)
print(accuracy)

0.7853810264385692


**Вывод:** Модель "Логистической регрессии" дала примерно такой же вариант,как и в моделе "Дерево решений". Надо отметить, что достаточно высокая доля предсказания. 

<div class="alert alert-block alert-info">
Итак, по результатам всех экспериментов, какую модель выбираем?
</div>

## 4. Проверка модели на тестовой выборке

In [13]:
#Проверяем выбранную нами модель "Случайного леса" на тестовой выборке
features_test = data_test.drop(['is_ultra'], axis=1)
target_test = data_test['is_ultra']

model = RandomForestClassifier(n_estimators=60, max_depth=10, random_state=rnd_state)
model.fit(features_train, target_train)

predictions_test = model.predict(features_test)
accuracy_score(target_test, predictions_test)

0.80248833592535

<div class="alert alert-block alert-danger">

~~Почему именно такие параметры n_estimators и max_depth?~~ 
</div> **ответила в выводе шага**

<div class="alert alert-block alert-success">
В остальном, проверка качества на тестовой выборке проведена корректно.
</div>

**Вывод:** С поставленной задачей справились, результат получился не ниже, чем 0.75. Явной проблемы с переобучением мы не наблюдаем, можно говорить о том, что модель для бинарной классификации, которая бы выбрала подходящий тариф мы подобрали. 

Выбрали именно модель "Случайный лес" с заданными параметрами, так как accuracy при таких параметрах наиболее высокий из всех рассмотренных моделей. 

## 5. Проверка модели на адекватность

In [14]:
data['is_ultra'].value_counts()

0    2229
1     985
Name: is_ultra, dtype: int64

In [15]:
data_test['is_ultra'].value_counts()

0    440
1    203
Name: is_ultra, dtype: int64

In [16]:
(data['is_ultra']==0).sum() / data.shape[0]

0.693528313627878

**Вывод:** Модель, которую мы протестировали предсказыывает лучше, чем то количество пришедших на тариф "Смарт" (к примеру) к общему количеству клиентов в представленной выборке. 
Модель можно назвать адекватной и можно использовать далее в работе, так как доля ее предсказания выше, чем предположение, которое мы моги бы построить. 

<div class="alert alert-block alert-success">
Проверка на адекватность выполнена правильно, молодец! 
    
Действительно, проверка на адекватность означает сравнение качества итоговой модели с качеством простой, базовой модели, например, предсказывающей для всех объектов тестовой выборки один и тот же класс «0» (или 1). Можешь посмотреть класс DummyClassifier, там реализованы такие простые модели предсказаний.

</div>

Влада, вечер добрый! Спасибо за твои комментарии, я постараюсь их учесть в будущем, а также подкорректировать все недочеты. Надеюсь, что сейчас все получилось! =)

<div class="alert alert-block alert-warning">
<b>Итоговый комментарий:</b> 

Спасибо, ты провела прекрасное исследование, так держать! ~~осталось его немного доработать.~~

</div>

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
